# NB10: Statistical Significance Tests

**Question:** Are the performance differences we observe statistically significant?

This notebook produces thesis-ready results:
1. **Paired bootstrap tests** for key comparisons (RAG vs Direct, reranker impact, etc.)
2. **Wilcoxon signed-rank tests** as non-parametric alternative
3. **Multiple comparison correction** (Holm-Bonferroni)
4. **LaTeX tables** for direct inclusion in thesis

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as scipy_stats
from itertools import combinations

from analysis_utils import (
    load_all_results, load_per_item_scores, setup_plotting,
    compute_per_question_rag_delta, effect_size,
    PRIMARY_METRIC, BROKEN_MODELS, MODEL_TIER, MODEL_PARAMS,
)

setup_plotting()
np.random.seed(42)

STUDY_PATH = Path("../outputs/smart_retrieval_slm")
ALPHA = 0.05  # Significance level
N_BOOTSTRAP = 10_000  # Bootstrap iterations

# Load experiment-level results
df_all = load_all_results(STUDY_PATH)
df = df_all[~df_all['model_short'].isin(BROKEN_MODELS)].copy()
df['tier'] = df['model_short'].map(MODEL_TIER)
df['params_b'] = df['model_short'].map(MODEL_PARAMS)

rag = df[df['exp_type'] == 'rag'].copy()
direct = df[df['exp_type'] == 'direct'].copy()

print(f"Total experiments: {len(df)} ({len(rag)} RAG, {len(direct)} direct)")
print(f"Models: {sorted(df['model_short'].unique())}")
print(f"Datasets: {sorted(df['dataset'].unique())}")

## Statistical Test Functions

Paired bootstrap and Wilcoxon tests with Holm-Bonferroni correction.

In [ ]:
def paired_bootstrap_test(a: np.ndarray, b: np.ndarray, n_bootstrap: int = N_BOOTSTRAP,
                          seed: int = 42) -> dict:
    """Paired bootstrap test: is mean(b) > mean(a)?

    Returns dict with observed_diff, ci_low, ci_high, p_value.
    """
    rng = np.random.default_rng(seed)
    assert len(a) == len(b), f"Paired test requires equal lengths: {len(a)} vs {len(b)}"

    observed_diff = np.mean(b) - np.mean(a)
    diffs = b - a

    boot_diffs = np.empty(n_bootstrap)
    for i in range(n_bootstrap):
        idx = rng.choice(len(diffs), size=len(diffs), replace=True)
        boot_diffs[i] = np.mean(diffs[idx])

    ci_low = np.percentile(boot_diffs, 2.5)
    ci_high = np.percentile(boot_diffs, 97.5)

    # Two-sided p-value: proportion of bootstrap samples on the other side of 0
    if observed_diff >= 0:
        p_value = np.mean(boot_diffs <= 0) * 2
    else:
        p_value = np.mean(boot_diffs >= 0) * 2
    p_value = min(p_value, 1.0)

    return {
        'observed_diff': observed_diff,
        'ci_low': ci_low,
        'ci_high': ci_high,
        'p_value': p_value,
        'n': len(a),
    }


def unpaired_bootstrap_test(a: np.ndarray, b: np.ndarray, n_bootstrap: int = N_BOOTSTRAP,
                            seed: int = 42) -> dict:
    """Unpaired bootstrap test: is mean(b) > mean(a)?"""
    rng = np.random.default_rng(seed)
    observed_diff = np.mean(b) - np.mean(a)

    boot_diffs = np.empty(n_bootstrap)
    for i in range(n_bootstrap):
        boot_a = rng.choice(a, size=len(a), replace=True)
        boot_b = rng.choice(b, size=len(b), replace=True)
        boot_diffs[i] = np.mean(boot_b) - np.mean(boot_a)

    ci_low = np.percentile(boot_diffs, 2.5)
    ci_high = np.percentile(boot_diffs, 97.5)

    if observed_diff >= 0:
        p_value = np.mean(boot_diffs <= 0) * 2
    else:
        p_value = np.mean(boot_diffs >= 0) * 2
    p_value = min(p_value, 1.0)

    return {
        'observed_diff': observed_diff,
        'ci_low': ci_low,
        'ci_high': ci_high,
        'p_value': p_value,
        'n_a': len(a),
        'n_b': len(b),
    }


def holm_bonferroni(p_values: list[float], alpha: float = ALPHA) -> list[dict]:
    """Apply Holm-Bonferroni correction to a list of p-values.

    Returns list of dicts with original_p, adjusted_p, significant.
    """
    n = len(p_values)
    indexed = sorted(enumerate(p_values), key=lambda x: x[1])

    results = [None] * n
    for rank, (orig_idx, p) in enumerate(indexed):
        adjusted = p * (n - rank)
        adjusted = min(adjusted, 1.0)
        results[orig_idx] = {
            'original_p': p,
            'adjusted_p': adjusted,
            'significant': adjusted < alpha,
        }

    return results


def significance_stars(p: float) -> str:
    """Return significance stars for a p-value."""
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return 'ns'


def format_ci(diff, ci_low, ci_high, decimals=3) -> str:
    """Format difference with CI for display."""
    return f"{diff:+.{decimals}f} [{ci_low:+.{decimals}f}, {ci_high:+.{decimals}f}]"


print("Statistical test functions defined.")

## 1. RAG vs Direct Baseline

For each model x dataset pair, compare the best RAG config against the best direct baseline.
Uses unpaired bootstrap since RAG and direct have different numbers of experiments.

In [ ]:
# RAG vs Direct: per model x dataset
rag_vs_direct_rows = []

for model in sorted(df['model_short'].unique()):
    for dataset in sorted(df['dataset'].unique()):
        d_vals = direct[(direct['model_short'] == model) & (direct['dataset'] == dataset)][PRIMARY_METRIC].dropna().values
        r_vals = rag[(rag['model_short'] == model) & (rag['dataset'] == dataset)][PRIMARY_METRIC].dropna().values

        if len(d_vals) < 1 or len(r_vals) < 2:
            continue

        result = unpaired_bootstrap_test(d_vals, r_vals)

        # Also compute Wilcoxon if enough samples
        if len(r_vals) >= 5:
            # Mann-Whitney U (unpaired non-parametric)
            u_stat, mwu_p = scipy_stats.mannwhitneyu(r_vals, d_vals, alternative='two-sided')
        else:
            mwu_p = np.nan

        d_cohen, _, interp = effect_size(d_vals, r_vals)

        rag_vs_direct_rows.append({
            'model': model,
            'tier': MODEL_TIER.get(model, 'unknown'),
            'dataset': dataset,
            'direct_mean': np.mean(d_vals),
            'rag_mean': np.mean(r_vals),
            'diff': result['observed_diff'],
            'ci_low': result['ci_low'],
            'ci_high': result['ci_high'],
            'p_boot': result['p_value'],
            'p_mwu': mwu_p,
            'cohens_d': d_cohen,
            'effect': interp,
            'n_direct': len(d_vals),
            'n_rag': len(r_vals),
        })

rvd = pd.DataFrame(rag_vs_direct_rows)

if not rvd.empty:
    # Apply Holm-Bonferroni correction
    corrections = holm_bonferroni(rvd['p_boot'].tolist())
    rvd['p_adjusted'] = [c['adjusted_p'] for c in corrections]
    rvd['sig'] = [significance_stars(c['adjusted_p']) for c in corrections]

    print("RAG vs Direct: Per Model x Dataset")
    print("=" * 80)
    display(rvd[['model', 'tier', 'dataset', 'direct_mean', 'rag_mean', 'diff',
                 'ci_low', 'ci_high', 'p_adjusted', 'sig', 'cohens_d', 'effect']].round(4))

    # Summary: how often does RAG significantly help?
    n_sig_help = ((rvd['sig'] != 'ns') & (rvd['diff'] > 0)).sum()
    n_sig_hurt = ((rvd['sig'] != 'ns') & (rvd['diff'] < 0)).sum()
    n_ns = (rvd['sig'] == 'ns').sum()
    print(f"\nRAG significantly helps: {n_sig_help}/{len(rvd)} combos")
    print(f"RAG significantly hurts: {n_sig_hurt}/{len(rvd)} combos")
    print(f"No significant difference: {n_ns}/{len(rvd)} combos")

In [ ]:
# Visualize RAG vs Direct effect sizes
if not rvd.empty:
    fig, ax = plt.subplots(figsize=(12, max(4, len(rvd) * 0.4)))

    rvd_sorted = rvd.sort_values('diff')
    y_labels = rvd_sorted['model'] + ' / ' + rvd_sorted['dataset']
    y_pos = range(len(rvd_sorted))

    colors = ['#2ecc71' if d > 0 and s != 'ns' else '#e74c3c' if d < 0 and s != 'ns' else '#95a5a6'
              for d, s in zip(rvd_sorted['diff'], rvd_sorted['sig'])]

    ax.barh(y_pos, rvd_sorted['diff'], color=colors, alpha=0.8, edgecolor='black', linewidth=0.5)
    ax.errorbar(rvd_sorted['diff'].values, y_pos,
                xerr=[rvd_sorted['diff'].values - rvd_sorted['ci_low'].values,
                      rvd_sorted['ci_high'].values - rvd_sorted['diff'].values],
                fmt='none', color='black', capsize=3, linewidth=1)

    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(y_labels, fontsize=9)
    ax.set_xlabel(f'Mean {PRIMARY_METRIC.upper()} Difference (RAG - Direct)')
    ax.set_title('RAG vs Direct: Effect Size with 95% Bootstrap CI\n'
                 '(green = significant positive, red = significant negative, grey = not significant)')
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 2. Reranker Impact

Pairwise comparisons: none vs bge vs bge-v2, controlling for model and dataset.

In [ ]:
# Reranker pairwise comparisons
reranker_levels = sorted(rag['reranker'].dropna().unique())
reranker_rows = []

for a_level, b_level in combinations(reranker_levels, 2):
    a_vals = rag[rag['reranker'] == a_level][PRIMARY_METRIC].dropna().values
    b_vals = rag[rag['reranker'] == b_level][PRIMARY_METRIC].dropna().values

    if len(a_vals) < 3 or len(b_vals) < 3:
        continue

    result = unpaired_bootstrap_test(a_vals, b_vals)
    d_cohen, _, interp = effect_size(a_vals, b_vals)

    reranker_rows.append({
        'baseline': a_level,
        'treatment': b_level,
        'baseline_mean': np.mean(a_vals),
        'treatment_mean': np.mean(b_vals),
        'diff': result['observed_diff'],
        'ci_low': result['ci_low'],
        'ci_high': result['ci_high'],
        'p_boot': result['p_value'],
        'cohens_d': d_cohen,
        'effect': interp,
        'n_baseline': len(a_vals),
        'n_treatment': len(b_vals),
    })

rr_df = pd.DataFrame(reranker_rows)
if not rr_df.empty:
    corrections = holm_bonferroni(rr_df['p_boot'].tolist())
    rr_df['p_adjusted'] = [c['adjusted_p'] for c in corrections]
    rr_df['sig'] = [significance_stars(c['adjusted_p']) for c in corrections]

    print("Reranker Pairwise Comparisons")
    print("=" * 80)
    display(rr_df[['baseline', 'treatment', 'baseline_mean', 'treatment_mean',
                   'diff', 'ci_low', 'ci_high', 'p_adjusted', 'sig',
                   'cohens_d', 'effect']].round(4))

## 3. Agent Type Comparisons

Pairwise: fixed_rag vs iterative_rag vs self_rag.

In [ ]:
agent_levels = sorted(rag['agent_type'].dropna().unique())
agent_rows = []

for a_level, b_level in combinations(agent_levels, 2):
    a_vals = rag[rag['agent_type'] == a_level][PRIMARY_METRIC].dropna().values
    b_vals = rag[rag['agent_type'] == b_level][PRIMARY_METRIC].dropna().values

    if len(a_vals) < 3 or len(b_vals) < 3:
        continue

    result = unpaired_bootstrap_test(a_vals, b_vals)
    d_cohen, _, interp = effect_size(a_vals, b_vals)

    agent_rows.append({
        'baseline': a_level,
        'treatment': b_level,
        'baseline_mean': np.mean(a_vals),
        'treatment_mean': np.mean(b_vals),
        'diff': result['observed_diff'],
        'ci_low': result['ci_low'],
        'ci_high': result['ci_high'],
        'p_boot': result['p_value'],
        'cohens_d': d_cohen,
        'effect': interp,
        'n_baseline': len(a_vals),
        'n_treatment': len(b_vals),
    })

agent_df = pd.DataFrame(agent_rows)
if not agent_df.empty:
    corrections = holm_bonferroni(agent_df['p_boot'].tolist())
    agent_df['p_adjusted'] = [c['adjusted_p'] for c in corrections]
    agent_df['sig'] = [significance_stars(c['adjusted_p']) for c in corrections]

    print("Agent Type Pairwise Comparisons")
    print("=" * 80)
    display(agent_df[['baseline', 'treatment', 'baseline_mean', 'treatment_mean',
                      'diff', 'ci_low', 'ci_high', 'p_adjusted', 'sig',
                      'cohens_d', 'effect']].round(4))

## 4. Query Transform Impact

Pairwise: none vs hyde vs multiquery.

In [ ]:
qt_levels = sorted(rag['query_transform'].dropna().unique())
qt_rows = []

for a_level, b_level in combinations(qt_levels, 2):
    a_vals = rag[rag['query_transform'] == a_level][PRIMARY_METRIC].dropna().values
    b_vals = rag[rag['query_transform'] == b_level][PRIMARY_METRIC].dropna().values

    if len(a_vals) < 3 or len(b_vals) < 3:
        continue

    result = unpaired_bootstrap_test(a_vals, b_vals)
    d_cohen, _, interp = effect_size(a_vals, b_vals)

    qt_rows.append({
        'baseline': a_level,
        'treatment': b_level,
        'baseline_mean': np.mean(a_vals),
        'treatment_mean': np.mean(b_vals),
        'diff': result['observed_diff'],
        'ci_low': result['ci_low'],
        'ci_high': result['ci_high'],
        'p_boot': result['p_value'],
        'cohens_d': d_cohen,
        'effect': interp,
        'n_baseline': len(a_vals),
        'n_treatment': len(b_vals),
    })

qt_df = pd.DataFrame(qt_rows)
if not qt_df.empty:
    corrections = holm_bonferroni(qt_df['p_boot'].tolist())
    qt_df['p_adjusted'] = [c['adjusted_p'] for c in corrections]
    qt_df['sig'] = [significance_stars(c['adjusted_p']) for c in corrections]

    print("Query Transform Pairwise Comparisons")
    print("=" * 80)
    display(qt_df[['baseline', 'treatment', 'baseline_mean', 'treatment_mean',
                   'diff', 'ci_low', 'ci_high', 'p_adjusted', 'sig',
                   'cohens_d', 'effect']].round(4))

## 5. Model Tier Comparisons

Tiny (1-2B) vs Small (3B) vs Medium (7-9B) on RAG experiments.

In [ ]:
rag_with_tier = rag.dropna(subset=['tier']).copy()
tier_levels = ['tiny', 'small', 'medium']
tier_rows = []

for a_tier, b_tier in combinations(tier_levels, 2):
    a_vals = rag_with_tier[rag_with_tier['tier'] == a_tier][PRIMARY_METRIC].dropna().values
    b_vals = rag_with_tier[rag_with_tier['tier'] == b_tier][PRIMARY_METRIC].dropna().values

    if len(a_vals) < 3 or len(b_vals) < 3:
        continue

    result = unpaired_bootstrap_test(a_vals, b_vals)
    d_cohen, _, interp = effect_size(a_vals, b_vals)

    tier_rows.append({
        'baseline': a_tier,
        'treatment': b_tier,
        'baseline_mean': np.mean(a_vals),
        'treatment_mean': np.mean(b_vals),
        'diff': result['observed_diff'],
        'ci_low': result['ci_low'],
        'ci_high': result['ci_high'],
        'p_boot': result['p_value'],
        'cohens_d': d_cohen,
        'effect': interp,
        'n_baseline': len(a_vals),
        'n_treatment': len(b_vals),
    })

tier_df = pd.DataFrame(tier_rows)
if not tier_df.empty:
    corrections = holm_bonferroni(tier_df['p_boot'].tolist())
    tier_df['p_adjusted'] = [c['adjusted_p'] for c in corrections]
    tier_df['sig'] = [significance_stars(c['adjusted_p']) for c in corrections]

    print("Model Tier Comparisons (RAG only)")
    print("=" * 80)
    display(tier_df[['baseline', 'treatment', 'baseline_mean', 'treatment_mean',
                     'diff', 'ci_low', 'ci_high', 'p_adjusted', 'sig',
                     'cohens_d', 'effect']].round(4))

    # Per-dataset breakdown
    print("\nTier comparisons per dataset:")
    for dataset in sorted(rag_with_tier['dataset'].unique()):
        ds_data = rag_with_tier[rag_with_tier['dataset'] == dataset]
        tier_means = ds_data.groupby('tier')[PRIMARY_METRIC].agg(['mean', 'count']).round(4)
        print(f"\n  {dataset}:")
        for tier_name in tier_levels:
            if tier_name in tier_means.index:
                m = tier_means.loc[tier_name]
                print(f"    {tier_name:<8s}: {m['mean']:.4f} (n={int(m['count'])})")

## 6. Best RAG vs Best Direct (Per Model x Dataset)

The most relevant thesis comparison: does the *best* RAG config beat the *best* direct config?

In [ ]:
best_comparison_rows = []

for model in sorted(df['model_short'].unique()):
    for dataset in sorted(df['dataset'].unique()):
        d_sub = direct[(direct['model_short'] == model) & (direct['dataset'] == dataset)]
        r_sub = rag[(rag['model_short'] == model) & (rag['dataset'] == dataset)]

        d_vals = d_sub[PRIMARY_METRIC].dropna()
        r_vals = r_sub[PRIMARY_METRIC].dropna()

        if d_vals.empty or r_vals.empty:
            continue

        best_direct = d_vals.max()
        best_rag = r_vals.max()

        # Top-5 RAG mean as a more robust estimate
        top5_rag = r_vals.nlargest(min(5, len(r_vals))).mean()

        best_comparison_rows.append({
            'model': model,
            'tier': MODEL_TIER.get(model, 'unknown'),
            'dataset': dataset,
            'best_direct': best_direct,
            'best_rag': best_rag,
            'top5_rag_mean': top5_rag,
            'rag_advantage': best_rag - best_direct,
            'rag_advantage_pct': (best_rag - best_direct) / best_direct * 100 if best_direct > 0 else np.nan,
            'n_rag_configs': len(r_vals),
        })

best_df = pd.DataFrame(best_comparison_rows)
if not best_df.empty:
    print("Best RAG vs Best Direct (per model x dataset)")
    print("=" * 80)
    display(best_df.round(4))

    n_rag_wins = (best_df['rag_advantage'] > 0).sum()
    print(f"\nRAG beats direct: {n_rag_wins}/{len(best_df)} combos")
    print(f"Mean RAG advantage: {best_df['rag_advantage'].mean():+.4f} F1")
    print(f"Mean RAG advantage: {best_df['rag_advantage_pct'].mean():+.1f}%")

    # By tier
    print("\nBy model tier:")
    for tier in ['tiny', 'small', 'medium']:
        t_df = best_df[best_df['tier'] == tier]
        if not t_df.empty:
            wins = (t_df['rag_advantage'] > 0).sum()
            print(f"  {tier}: RAG wins {wins}/{len(t_df)}, "
                  f"mean advantage {t_df['rag_advantage'].mean():+.4f}")

## 7. Consolidated Summary Table

In [ ]:
# Build a consolidated summary of all comparisons
summary_rows = []

# RAG vs Direct overall
if not rvd.empty:
    summary_rows.append({
        'comparison': 'RAG vs Direct (all)',
        'mean_diff': rvd['diff'].mean(),
        'n_significant': (rvd['sig'] != 'ns').sum(),
        'n_total': len(rvd),
        'median_d': rvd['cohens_d'].median(),
        'direction': 'RAG better' if rvd['diff'].mean() > 0 else 'Direct better',
    })

# Reranker: none vs bge-v2
if not rr_df.empty:
    for _, row in rr_df.iterrows():
        summary_rows.append({
            'comparison': f'Reranker: {row["treatment"]} vs {row["baseline"]}',
            'mean_diff': row['diff'],
            'n_significant': 1 if row['sig'] != 'ns' else 0,
            'n_total': 1,
            'median_d': row['cohens_d'],
            'direction': f'{row["treatment"]} better' if row['diff'] > 0 else f'{row["baseline"]} better',
        })

# Agent types
if not agent_df.empty:
    for _, row in agent_df.iterrows():
        summary_rows.append({
            'comparison': f'Agent: {row["treatment"]} vs {row["baseline"]}',
            'mean_diff': row['diff'],
            'n_significant': 1 if row['sig'] != 'ns' else 0,
            'n_total': 1,
            'median_d': row['cohens_d'],
            'direction': f'{row["treatment"]} better' if row['diff'] > 0 else f'{row["baseline"]} better',
        })

# Query transform
if not qt_df.empty:
    for _, row in qt_df.iterrows():
        summary_rows.append({
            'comparison': f'QT: {row["treatment"]} vs {row["baseline"]}',
            'mean_diff': row['diff'],
            'n_significant': 1 if row['sig'] != 'ns' else 0,
            'n_total': 1,
            'median_d': row['cohens_d'],
            'direction': f'{row["treatment"]} better' if row['diff'] > 0 else f'{row["baseline"]} better',
        })

# Model tiers
if not tier_df.empty:
    for _, row in tier_df.iterrows():
        summary_rows.append({
            'comparison': f'Tier: {row["treatment"]} vs {row["baseline"]}',
            'mean_diff': row['diff'],
            'n_significant': 1 if row['sig'] != 'ns' else 0,
            'n_total': 1,
            'median_d': row['cohens_d'],
            'direction': f'{row["treatment"]} better' if row['diff'] > 0 else f'{row["baseline"]} better',
        })

summary_df = pd.DataFrame(summary_rows)
if not summary_df.empty:
    print("Consolidated Significance Summary")
    print("=" * 80)
    display(summary_df.round(4))

## 8. LaTeX Tables

Publication-ready tables for direct inclusion in the thesis.

In [ ]:
def df_to_latex(df, columns, col_names=None, caption="", label="",
                fmt=None, bold_col=None, bold_fn=None):
    """Convert a DataFrame to a LaTeX table string.

    Args:
        df: DataFrame to convert.
        columns: List of column names to include.
        col_names: Display names for columns (same length as columns).
        caption: LaTeX table caption.
        label: LaTeX table label.
        fmt: Dict mapping column names to format strings.
        bold_col: Column to apply bold formatting to.
        bold_fn: Function(value) -> bool; if True, bold that cell.
    """
    if col_names is None:
        col_names = columns
    if fmt is None:
        fmt = {}

    n_cols = len(columns)
    col_spec = 'l' + 'r' * (n_cols - 1)

    lines = [
        r'\begin{table}[htbp]',
        r'\centering',
        f'\\caption{{{caption}}}',
        f'\\label{{{label}}}',
        f'\\begin{{tabular}}{{{col_spec}}}',
        r'\toprule',
        ' & '.join(f'\\textbf{{{n}}}' for n in col_names) + r' \\',
        r'\midrule',
    ]

    for _, row in df.iterrows():
        cells = []
        for col in columns:
            val = row[col]
            if col in fmt and isinstance(val, (int, float)) and not pd.isna(val):
                cell = fmt[col].format(val)
            else:
                cell = str(val)
            # Escape underscores for LaTeX
            cell = cell.replace('_', r'\_')
            if bold_col and col == bold_col and bold_fn and bold_fn(val):
                cell = f'\\textbf{{{cell}}}'
            cells.append(cell)
        lines.append(' & '.join(cells) + r' \\')

    lines.extend([
        r'\bottomrule',
        r'\end{tabular}',
        r'\end{table}',
    ])

    return '\n'.join(lines)


# Table 1: RAG vs Direct per Model x Dataset
if not rvd.empty:
    latex1 = df_to_latex(
        rvd.sort_values(['tier', 'model', 'dataset']),
        columns=['model', 'dataset', 'direct_mean', 'rag_mean', 'diff', 'p_adjusted', 'sig', 'effect'],
        col_names=['Model', 'Dataset', 'Direct F1', 'RAG F1', '$\\Delta$', '$p_{adj}$', 'Sig.', 'Effect'],
        caption='RAG vs Direct LLM performance per model and dataset. '
                'P-values corrected with Holm-Bonferroni. Effect size: Cohen\'s d.',
        label='tab:rag-vs-direct',
        fmt={'direct_mean': '{:.3f}', 'rag_mean': '{:.3f}', 'diff': '{:+.3f}',
             'p_adjusted': '{:.4f}'},
        bold_col='sig',
        bold_fn=lambda v: v != 'ns',
    )
    print("% === Table 1: RAG vs Direct ===")
    print(latex1)
    print()

# Table 2: Component Impact Summary
component_rows = []
for label, comp_df in [('Reranker', rr_df), ('Agent Type', agent_df),
                       ('Query Transform', qt_df), ('Model Tier', tier_df)]:
    if comp_df is not None and not comp_df.empty:
        for _, row in comp_df.iterrows():
            component_rows.append({
                'component': label,
                'comparison': f"{row['treatment']} vs {row['baseline']}",
                'diff': row['diff'],
                'ci': f"[{row['ci_low']:+.3f}, {row['ci_high']:+.3f}]",
                'p_adjusted': row['p_adjusted'],
                'sig': row['sig'],
                'cohens_d': row['cohens_d'],
                'effect': row['effect'],
            })

comp_summary = pd.DataFrame(component_rows)
if not comp_summary.empty:
    latex2 = df_to_latex(
        comp_summary,
        columns=['component', 'comparison', 'diff', 'ci', 'p_adjusted', 'sig', 'effect'],
        col_names=['Component', 'Comparison', '$\\Delta$ F1', '95\\% CI',
                   '$p_{adj}$', 'Sig.', 'Effect'],
        caption='Pairwise component comparisons with bootstrap significance tests. '
                'All p-values Holm-Bonferroni corrected within each component group.',
        label='tab:component-significance',
        fmt={'diff': '{:+.3f}', 'p_adjusted': '{:.4f}', 'cohens_d': '{:.2f}'},
        bold_col='sig',
        bold_fn=lambda v: v != 'ns',
    )
    print("% === Table 2: Component Significance ===")
    print(latex2)
    print()

# Table 3: Best RAG vs Best Direct
if not best_df.empty:
    latex3 = df_to_latex(
        best_df.sort_values(['tier', 'model', 'dataset']),
        columns=['model', 'tier', 'dataset', 'best_direct', 'best_rag',
                 'rag_advantage', 'rag_advantage_pct'],
        col_names=['Model', 'Tier', 'Dataset', 'Best Direct', 'Best RAG',
                   '$\\Delta$ F1', '$\\Delta$ \\%'],
        caption='Best RAG configuration vs best direct LLM per model and dataset.',
        label='tab:best-rag-vs-direct',
        fmt={'best_direct': '{:.3f}', 'best_rag': '{:.3f}',
             'rag_advantage': '{:+.3f}', 'rag_advantage_pct': '{:+.1f}'},
        bold_col='rag_advantage',
        bold_fn=lambda v: isinstance(v, (int, float)) and v > 0,
    )
    print("% === Table 3: Best RAG vs Best Direct ===")
    print(latex3)

## 9. Summary

Key findings from statistical tests:
- Whether RAG significantly outperforms direct LLM (per model tier and dataset)
- Which components have statistically significant effects (reranker, agent, query transform)
- Effect sizes (Cohen's d) to quantify practical significance
- LaTeX tables ready for thesis inclusion